# Fuzzy-Fairness DSS for LEO Satellites - Interactive Demo

This notebook provides an interactive demonstration of the fuzzy-fairness dynamic spectrum sharing system.


In [ ]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '..'))

import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

from src.dss import DSSSimulator
from src.fairness import FuzzyInferenceSystem, FairnessEvaluator
from src.visualization import plot_fairness_radar, plot_fairness_over_time


## 1. Initialize Simulator


In [ ]:
# Create simulator
simulator = DSSSimulator(
    ground_stations=[(40.0, -100.0)],
    frequency_range_hz=(10e9, 12e9),
    policy_type='fuzzy'
)

# Add users
simulator.add_user('user_1', demand=0.8, priority=0.7)
simulator.add_user('user_2', demand=0.6, priority=0.5)
simulator.add_user('user_3', demand=0.4, priority=0.3)

print(f"Added {len(simulator.users)} users")


## 2. Run Simulation


In [ ]:
# Run simulation
start_time = datetime.now()
end_time = start_time + timedelta(hours=1)

results = simulator.run(start_time, end_time, time_step_seconds=60.0)

print(f"Simulation completed: {len(results['results'])} time steps")


## 3. Visualize Results


In [ ]:
# Plot fairness over time
fairness_history = results['fairness_history']
plot_fairness_over_time(
    fairness_history,
    metric_name='fuzzy_fairness_score',
    title='Fuzzy Fairness Score Over Time'
)
plt.show()


In [ ]:
# Final fairness radar chart
if len(fairness_history) > 0:
    final_metrics = fairness_history[-1]
    radar_metrics = {
        'Jain Index': final_metrics.get('jain_index', 0),
        '1 - Gini': 1 - final_metrics.get('gini_coefficient', 1),
        'Fuzzy Score': final_metrics.get('fuzzy_fairness_score', 0),
        'Max-Min': final_metrics.get('max_min_fairness', 0) / max(final_metrics.get('mean_allocation', 1), 1)
    }
    
    plot_fairness_radar(
        radar_metrics,
        title='Final Fairness Metrics'
    )
    plt.show()


## 4. Explore Fuzzy Inference System


In [ ]:
# Test fuzzy inference
fis = FuzzyInferenceSystem()

# Test different input combinations
test_inputs = [
    {'load': 0.2, 'fairness': 0.5, 'priority': 0.5},  # Light load
    {'load': 0.5, 'fairness': 0.5, 'priority': 0.5},  # Moderate load
    {'load': 0.9, 'fairness': 0.3, 'priority': 0.5},  # Heavy load
]

for inputs in test_inputs:
    output = fis.infer(inputs)
    explanation = fis.explain_inference(inputs)
    print(f"Inputs: {inputs}")
    print(f"Fuzzy Score: {output:.3f}")
    print(f"Active Rules: {len(explanation['active_rules'])}")
    print()
